In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
df.drop("RID",axis=1, inplace = True) #RID는 그냥 Index라서 삭제

In [2]:
df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## 함수 만들기

In [3]:
def getEntropy(df, feature) :
    entropy = 0 #초기값 0으로 지정
    total = len(df[feature]) #전체 행 개수
    number = df[feature].value_counts() #타겟변수 class별 개수 파악
    for i in number:
        p = i/total #class별 데이터 개수/전체 행 개수를 p로 지정하고
        entropy -= p * np.log2(p) #엔트로피 공식에 맞춰 entropy에서 빼주기
    
    return(entropy)

In [4]:
getEntropy(df, "class_buys_computer")

0.9402859586706311

In [16]:
def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수 Feature에 대한 Info(Entropy)를 구한다.
    columns_name = list(df.loc[:, df.columns != feature]) # 목표변수를 제외한 나머지 설명변수들을 리스트 형태로 저장한다.
    
    
    kio = [] # gain값을 담을 빈 리스트 생성
    for i in columns_name: #목표변수를 제외한 나머지 변수를 하나씩 불러들이기
        new_entropy = 0 #info(D_i) 초기값 0으로 지정
        for j in list(df[i].unique()): #각 변수의 class를 하나씩 불러들이기
            entropy = 0 #엔트로피 초기값지정
            entropy += getEntropy(df.loc[df[i] == j],feature) #해당하는 class를 담고 있는 데이터프레임을 추출하여 해당하는 엔트로피값을 구해 차례로 더해준다(ex. age변수의 'youth')
            new_entropy += len(df.loc[df[i] == j])/len(df) * entropy #해당하는 class 데이터 행 개수/전체 데이터 행 개수 * 해당 feature의 class의 엔트로피값을 new_entropy에 더해주어 저장해놓는다
        kio_entropy = info_D - new_entropy #gain = info(D) - Info(D_i)
        kio.append(kio_entropy) #변수별로 구해준 gain값을 kio리스트에 저장 
            

    result = dict(zip(columns,kio)) # 각 변수에 대한 Information Gain 을 Dictionary 형태로 저장한다.
    return(result)

In [17]:
getGainA(df, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

## 결과 확인하기

In [18]:
my_dict = getGainA(df, "class_buys_computer")
def f1(x):
    return my_dict[x]
key_max = max(my_dict.keys(), key=f1)
print('정보 획득이 가장 높은 변수는',key_max, "이며 정보 획득량은", my_dict[key_max], "이다.")

정보 획득이 가장 높은 변수는 age 이며 정보 획득량은 0.24674981977443933 이다.
